In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# EPL Match Stat Exploration

Data exploration of the newly created database for only the English Premier League

Check what data we have, if it squares with results elsewhere and build some core funtions for manipulating the data efficiently

## 1. Imports

In [2]:
# mixture of libs for web scraping, parsing and pandas
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import warnings

import epl.query

pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

Import the data using a SQL query

In [3]:
DATA_PATH = '../data/match_results.sqlite'
try:
    conn = sqlite3.connect(DATA_PATH)
    print('Connection established')
except:
    print('Could not establish connection')

Connection established


In [4]:
query = """ SELECT *
            FROM matches
            WHERE Div='E0'
            """

df_epl = pd.read_sql(query, conn)
df_epl['Date'] = pd.to_datetime(df_epl.Date)
df_epl.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,country,league,season,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA,SJH,SJD,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA,GB>2.5,GB<2.5,GBAHH,GBAHA,GBAH,LBAHH,LBAHA,LBAH,B365AH,SOH,SOD,SOA,Attendance,HHW,AHW,HO,AO,HBP,ABP,SYH,SYD,SYA
0,E0,2020-09-12,12:30,Fulham,Arsenal,0.0,3.0,A,0.0,1.0,A,C Kavanagh,5.0,13.0,2.0,6.0,12.0,12.0,2.0,3.0,2.0,2.0,0.0,0.0,6.00,4.33,1.53,5.50,4.25,1.57,6.00,3.90,1.57,6.16,4.51,1.56,6.50,4.2,1.53,6.50,4.20,1.55,6.55,4.55,1.60,5.94,4.34,1.55,1.72,2.10,1.80,2.13,1.84,2.18,1.76,2.10,1.00,1.93,1.97,1.96,1.96,2.00,1.99,1.93,1.95,5.00,4.00,1.66,5.50,4.00,1.62,5.25,3.90,1.67,5.48,3.98,1.69,5.50,3.8,1.65,5.50,3.90,1.67,5.75,4.20,1.71,5.36,3.93,1.67,2.0,1.80,2.06,1.86,2.10,1.92,2.00,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87,england,Premier League,2020/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1.0,0.0,H,1.0,0.0,H,Jj Moss,5.0,9.0,3.0,5.0,14.0,11.0,7.0,3.0,2.0,1.0,0.0,0.0,3.10,3.25,2.37,3.00,3.20,2.45,3.15,2.95,2.40,3.32,3.29,2.40,3.20,3.2,2.35,3.20,3.20,2.40,3.36,3.36,2.50,3.18,3.22,2.39,2.20,1.66,2.34,1.68,2.36,1.73,2.24,1.67,0.25,1.85,2.05,1.88,2.05,1.88,2.07,1.84,2.03,3.00,3.25,2.40,3.00,3.30,2.40,3.05,2.90,2.45,3.09,3.27,2.54,3.10,3.1,2.45,3.10,3.25,2.45,3.25,3.33,2.55,3.08,3.22,2.47,2.2,1.66,2.26,1.72,2.27,1.78,2.18,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12,england,Premier League,2020/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2020-09-12,17:30,Liverpool,Leeds,4.0,3.0,H,3.0,2.0,H,M Oliver,22.0,6.0,6.0,3.0,9.0,6.0,9.0,0.0,1.0,0.0,0.0,0.0,1.28,6.00,9.50,1.26,6.25,10.50,1.35,5.00,8.50,1.31,6.25,9.92,1.27,6.0,10.00,1.30,5.75,10.50,1.35,6.50,10.75,1.30,5.96,9.68,1.53,2.50,1.56,2.60,1.56,2.68,1.52,2.53,-1.50,1.95,1.95,1.97,1.95,2.00,2.08,1.90,1.97,1.25,6.00,11.00,1.25,6.25,11.00,1.30,6.00,9.00,1.28,6.34,11.38,1.25,6.0,12.00,1.29,6.00,11.50,1.30,6.75,12.27,1.28,6.16,10.63,1.5,2.62,1.51,2.76,1.53,2.82,1.50,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04,england,Premier League,2020/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2020-09-12,20:00,West Ham,Newcastle,0.0,2.0,A,0.0,0.0,D,S Attwell,15.0,15.0,3.0,2.0,13.0,7.0,8.0,7.0,2.0,2.0,0.0,0.0,2.15,3.40,3.40,2.15,3.40,3.40,2.15,3.15,3.40,2.18,3.61,3.50,2.15,3.5,3.40,2.15,3.40,3.60,2.24,3.70,3.60,2.15,3.48,3.42,1.90,1.90,2.00,1.91,2.05,1.95,1.97,1.86,-0.50,2.07,1.72,2.17,1.78,2.17,1.81,2.12,1.75,1.95,3.60,3.75,1.95,3.70,3.75,2.05,3.25,3.75,2.04,3.59,3.92,2.00,3.5,3.80,2.00,3.50,3.90,2.07,3.78,3.99,2.01,3.57,3.79,1.9,1.90,2.00,1.92,2.00,2.05,1.91,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86,england,Premier League,2020/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Quick check to see how many games we have results for

In [5]:
df_epl[['season', 'HomeTeam', 'FTHG', 'FTAG', ]].groupby('season').agg({'HomeTeam': 'count', 'FTHG': 'sum', 'FTAG': 'sum'})

,HomeTeam,FTHG,FTAG
season,,,
1993/1994,462,663.0,532.0
1994/1995,462,697.0,498.0
1995/1996,380,580.0,408.0
1996/1997,380,559.0,411.0
1997/1998,380,592.0,427.0
1998/1999,380,553.0,406.0
1999/1900,380,635.0,425.0
2000/2001,380,587.0,405.0
2001/2002,380,557.0,444.0


So we have pretty complete data back to 1993 with a slight blip in 2003-2005 where we seem to be missing the results for 45 games

## 2. Build Exploration Functions

The data is in a certain format - home vs away

All matches involve 2 teams, however for a lot of stats we will wish to condense this to be agnostic of home vs away

We want the most efficient way to aggregate to avoid unnecessary joins or slicing

### 2a. For Any Season, Construct League Table asof Date

Stats we need to comprise the table:
 - matches played: Simple count
 - wins, losses and draws: Again count FTR by both H and A team
 - finishing points: Computed based on W L D
 - GF, GA, GD: Sum over games

In [6]:
# input data - the season and what date to get games before
season = '2018/2019'
date = dt.date.today()

In [7]:
# columns we need to construct the league table data
table_cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

In [8]:
# get the relevant games
df_table = df_epl[(df_epl.season == season) & (df_epl.Date <= pd.to_datetime(date))][table_cols]
df_table

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
418,Man United,Leicester,2.0,1.0,H
419,Bournemouth,Cardiff,2.0,0.0,H
420,Fulham,Crystal Palace,0.0,2.0,A
421,Huddersfield,Chelsea,0.0,3.0,A
422,Newcastle,Tottenham,1.0,2.0,A
...,...,...,...,...,...
793,Liverpool,Wolves,2.0,0.0,H
794,Man United,Cardiff,0.0,2.0,A
795,Southampton,Huddersfield,1.0,1.0,D
796,Tottenham,Everton,2.0,2.0,D


In [9]:
def result_calculator(match_results, res_type):
    """
    Function to output the league table for a set of matches including MP, GF, GA and points
    match_results: dataframe of match results including home and away team cols and score lines
    res_type: string of 'H' or 'A' - computes the results from a certain perspective
    """
    # check if we have the columns we need
    req_cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    for col in req_cols:
        if col not in match_results.columns:
            return 'Missing column: {}, need following cols: {}'.format(col, req_cols)
    # handle whether perspective of H or A
    if res_type == 'H':
        # make everything from H perspective
        match_results = match_results.rename(columns={'HomeTeam': 'Team', 'AwayTeam': 'Opp', 'FTHG': 'GF', 'FTAG': 'GA'})
        # compute points from H perspective
        home_p = {'H': 3, 'A': 0, 'D': 1}
        home_res = {'H': 'W', 'A': 'L', 'D': 'D'}
        
        match_results['Points'] = match_results['FTR'].map(home_p)
        match_results['FTR'] = match_results['FTR'].map(home_res)
        
    elif res_type == 'A':
        # make everything from A perspective
        match_results = match_results.rename(columns={'AwayTeam': 'Team', 'HomeTeam': 'Opp', 'FTHG': 'GA', 'FTAG': 'GF'})
        # compute points from A perspective
        away_p = {'A': 3, 'H': 0, 'D': 1}
        away_res = {'A': 'W', 'H': 'L', 'D': 'D'}
        
        match_results['Points'] = match_results['FTR'].map(away_p)
        match_results['FTR'] = match_results['FTR'].map(away_res)
    else:
        return 'res_type must either be H or A, not: {}'.format(res_type)
    
    return match_results.dropna()

In [10]:
def table_calculator(match_results, res_type):
    
    # compute from perspective we care about
    df_match = result_calculator(match_results, res_type)
    
    # agg by team and result
    df_match = df_match.groupby(['Team', 'FTR']).agg({'Opp': 'count', 'GF': 'sum', 'GA': 'sum', 'Points': 'sum'}).reset_index()
    df_match = df_match.rename(columns={'Opp': 'MP'})
    
    # pivot by W/L/D
    df_res = pd.pivot_table(data=df_match[['Team', 'FTR', 'MP']], index='Team', columns='FTR', values='MP').fillna(0)
    
    # 
    df_res_goals = df_match.groupby('Team').sum()
    df_res_goals['GD'] = df_res_goals['GF'] - df_res_goals['GA']
    df_res = pd.merge(left=df_res_goals, right=df_res, how='left', on='Team').sort_values('Points', ascending=False)
    df_res['Loc'] = res_type
    
    df_res = df_res[['MP', 'W', 'L', 'D', 'GF', 'GA', 'GD', 'Points']]
    
    return df_res

In [11]:
def full_table_calculator(match_results):
    
    df_home = table_calculator(match_results, 'H')
    df_away = table_calculator(match_results, 'A')
    
    df_res = pd.concat([df_home, df_away])
    df_res = df_res.groupby('Team').sum().sort_values(['Points', 'GD', 'GF'], ascending=False)
    df_res = df_res.reset_index().reset_index().rename(columns={'index': 'position'}).set_index('Team')
    df_res['position'] = df_res['position'] + 1
    df_res = df_res[[x for x in df_res.columns if x != 'position'] + ['position']]
    
    return df_res

In [12]:
def league_table_asof(div, season, asof_date, conn):
    
    # variable checking and error messages
    if not isinstance(div, str):
        try:
            elig_divs = pd.read_sql("""SELECT DISTINCT Div from matches""", conn)
            elig_divs = elig_divs['Div'].values
        except:
            return 'Cannot connect to db'
        conn.close()
        return "Div: {} not in db, must be from: {}".format(div, ", ".join(elig_divs))
    
    if not isinstance(season, str):
        try:
            elig_seasons = pd.read_sql("""SELECT DISTINCT season from matches""", conn)
            elig_seasons = elig_seasons['season'].values
        except:
            return 'Cannot connect to db'
        conn.close()
        return "Season: {} not in db, must be from: {}".format(season, ", ".join(elig_seasons))
    
    if not asof_date:
        asof_date = dt.date.today() + dt.timedelta(days=365*10)
        asof_date = pd.to_datetime(asof_date)
    else:
        if not isinstance(asof_date, pd.Timestamp):
            try:
                asof_date = pd.to_datetime(asof_date)
            except:
                return "Failed to convert asof_date to datetime using pd.to_datetime"
    
    # query required data from db
    table_cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'] + ['Date']
    query = """ SELECT {}
                FROM matches
                WHERE Div='{}'
                AND season='{}'
                """.format(", ".join(table_cols), div, season)

    df_raw = pd.read_sql(query, conn)
    df_raw['Date'] = pd.to_datetime(df_raw['Date'])
    
    df = df_raw[df_raw.Date <= asof_date]
    
    df_res = full_table_calculator(df)
    
    return df_res

In [14]:
div = 'E0'
season = '2019/2020'
asof_date = dt.date(2020,9,1)
conn = sqlite3.connect(DATA_PATH)

x = league_table_asof(div, season, asof_date, conn)
x

,MP,W,L,D,GF,GA,GD,Points,position
Team,,,,,,,,,
Liverpool,38,32.0,3.0,3.0,85.0,33.0,52.0,99,1
Man City,38,26.0,9.0,3.0,102.0,35.0,67.0,81,2
Man United,38,18.0,8.0,12.0,66.0,36.0,30.0,66,3
Chelsea,38,20.0,12.0,6.0,69.0,54.0,15.0,66,4
Leicester,38,18.0,12.0,8.0,67.0,41.0,26.0,62,5
Tottenham,38,16.0,11.0,11.0,61.0,47.0,14.0,59,6
Wolves,38,15.0,9.0,14.0,51.0,40.0,11.0,59,7
Arsenal,38,14.0,10.0,14.0,56.0,48.0,8.0,56,8
Sheffield United,38,14.0,12.0,12.0,39.0,39.0,0.0,54,9


Good result - for any league, any season at any time point in the season we can now query in around 150ms

### 2b. Compare Half Time vs Full Time Distributions

Now we want to analyse what the full time score line looks like conditional on the half time score line

In [15]:
def query_creator(table, cols=None, wc=None):
    
    if wc:
        conds = []
        for col,cond in wc.items():
            if isinstance(cond[1], str):
                conds.append("{} {} '{}'".format(col, cond[0], cond[1]))
            else:
                conds.append("{} {} {}".format(col, cond[0], cond[1]))
        wc = ' AND '.join(conds)
    else:
        wc = ''
    
    if cols:
        col_query = ', '.join(cols)
    else:
        col_query = '*'
    
    query = 'SELECT {} FROM {} WHERE {}'.format(', '.join(cols), table, wc)
    return query

In [16]:
def query_db(query):
    
    try:
        conn = sqlite3.connect(DATA_PATH)
    except:
        return "Unable to establish connection to {}".format(DATA_PATH)
    
    try:
        print('Running query: {}'.format(query))
        res = pd.read_sql(query, conn)
    except:
        return "Unable to run query: {}".format(query)
    
    return res

In [17]:
def create_and_query(table, cols=None, wc=None):
    
    query = query_creator(table, cols, wc)
    res = query_db(query)
    return res

In [18]:
def find_matches_by_score(score, is_ht=False, div=None, home_team=None, away_team=None, leading_team=None, losing_team=None):
    
    # form the where statement in the sql query as sql 10x-50x faster at filtering than pandas
    wc = {}
    if div:
        wc['Div'] = ['=', div]
    if home_team:
        wc['HomeTeam'] = ['=', home_team]
    if away_team:
        wc['AwayTeam'] = ['=', away_team]
    if len(wc) == 0:
        wc = None
    
    # get cols we care about
    cols = ['Div', 'Date', 'season', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG']
    # query
    df = create_and_query('matches', cols, wc).dropna()
    
    home_goals = 'HTHG' if is_ht else 'FTHG'
    away_goals = 'HTAG' if is_ht else 'FTAG'
    
    # create tuple for score and select where matches
    df['score'] = list(zip(df[home_goals], df[away_goals]))
    df = df[(df['score'] == score) | (df['score'] == score[::-1])]
    
    # if leading / trailing team specified then apply that filter
    # don't know how to do this in sql yet so easier in pandas for now post sql query
    if leading_team:
        if score[0] == score[1]:
            df = df[(df.HomeTeam == leading_team) | (df.AwayTeam == leading_team)]
        else:
            df = df[((df.HomeTeam == leading_team) & (df[home_goals] > df[away_goals])) | ((df.AwayTeam == leading_team) & (df[home_goals] < df[away_goals]))]
    if losing_team:
        if score[0] == score[1]:
            df = df[(df.HomeTeam == losing_team) | (df.AwayTeam == losing_team)]
        else:
            df = df[((df.HomeTeam == losing_team) & (df[home_goals] < df[away_goals])) | ((df.AwayTeam == leading_team) & (df[home_goals] > df[away_goals]))]
    
    
    
    return df

In [19]:
score = (3,1)
is_ht=False
div='E0'
leading_team = None
losing_team = None

x = find_matches_by_score(score, is_ht=is_ht, div=div, leading_team=leading_team, losing_team=losing_team)
x

Running query: SELECT Div, Date, season, HomeTeam, AwayTeam, FTHG, FTAG, HTHG, HTAG FROM matches WHERE Div = 'E0'


,Div,Date,season,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,score
6,E0,2020-09-14 00:00:00,2020/2021,Brighton,Chelsea,1.0,3.0,0.0,1.0,"(1.0, 3.0)"
10,E0,2020-09-19 00:00:00,2020/2021,Man United,Crystal Palace,1.0,3.0,0.0,1.0,"(1.0, 3.0)"
17,E0,2020-09-21 00:00:00,2020/2021,Wolves,Man City,1.0,3.0,0.0,2.0,"(1.0, 3.0)"
27,E0,2020-09-28 00:00:00,2020/2021,Liverpool,Arsenal,3.0,1.0,2.0,1.0,"(3.0, 1.0)"
31,E0,2020-10-03 00:00:00,2020/2021,Newcastle,Burnley,3.0,1.0,1.0,0.0,"(3.0, 1.0)"
...,...,...,...,...,...,...,...,...,...,...
9389,E0,1996-04-05 00:00:00,1995/1996,Middlesbrough,Sheffield Weds,3.0,1.0,0.0,0.0,"(3.0, 1.0)"
9402,E0,1996-04-08 00:00:00,1995/1996,Leeds,Nott'm Forest,1.0,3.0,1.0,2.0,"(1.0, 3.0)"
9405,E0,1996-04-08 00:00:00,1995/1996,QPR,Everton,3.0,1.0,2.0,0.0,"(3.0, 1.0)"
9414,E0,1996-04-13 00:00:00,1995/1996,Southampton,Man United,3.0,1.0,3.0,0.0,"(3.0, 1.0)"


In [20]:
# get all the prem data we care about
df = create_and_query('matches', ['season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG' ,'FTAG'], {'Div': ['=', 'E0']}).dropna()
df

Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG FROM matches WHERE Div = 'E0'


,season,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,2020/2021,2020-09-12 00:00:00,Fulham,Arsenal,0.0,3.0
1,2020/2021,2020-09-12 00:00:00,Crystal Palace,Southampton,1.0,0.0
2,2020/2021,2020-09-12 00:00:00,Liverpool,Leeds,4.0,3.0
3,2020/2021,2020-09-12 00:00:00,West Ham,Newcastle,0.0,2.0
4,2020/2021,2020-09-13 00:00:00,West Brom,Leicester,0.0,3.0
...,...,...,...,...,...,...
10367,1993/1994,1994-05-07 00:00:00,Sheffield Weds,Man City,1.0,1.0
10368,1993/1994,1994-05-07 00:00:00,Swindon,Leeds,0.0,5.0
10369,1993/1994,1994-05-07 00:00:00,Tottenham,QPR,1.0,2.0
10370,1993/1994,1994-05-07 00:00:00,West Ham,Southampton,3.0,3.0


In [21]:
# get all the seasons so we can map any season to the previous season to be able to lj the data on
seasons = df.season.unique()

# now create the dictionary mapping season to champ and prev season
# first we get the league table for each season
div = 'E0'
conn = sqlite3.connect(DATA_PATH)
league_tables = {}

for s in seasons:
    x = league_table_asof(div, s, None, conn)
    x['season'] = s
    league_tables[s] = x.reset_index()

In [22]:
s_to_next_s = dict(zip(seasons[1:], seasons[:-1]))

In [23]:
all_results = pd.concat(league_tables.values())
all_results['next_season'] = all_results['season'].map(s_to_next_s)
all_results = all_results.dropna()
all_results

,Team,MP,W,L,D,GF,GA,GD,Points,position,season,next_season
0,Liverpool,38,32.0,3.0,3.0,85.0,33.0,52.0,99,1,2019/2020,2020/2021
1,Man City,38,26.0,9.0,3.0,102.0,35.0,67.0,81,2,2019/2020,2020/2021
2,Man United,38,18.0,8.0,12.0,66.0,36.0,30.0,66,3,2019/2020,2020/2021
3,Chelsea,38,20.0,12.0,6.0,69.0,54.0,15.0,66,4,2019/2020,2020/2021
4,Leicester,38,18.0,12.0,8.0,67.0,41.0,26.0,62,5,2019/2020,2020/2021
...,...,...,...,...,...,...,...,...,...,...,...,...
17,Southampton,42,12.0,23.0,7.0,49.0,66.0,-17.0,43,18,1993/1994,1994/1995
18,Ipswich,42,9.0,17.0,16.0,35.0,58.0,-23.0,43,19,1993/1994,1994/1995
19,Sheffield United,42,8.0,16.0,18.0,42.0,60.0,-18.0,42,20,1993/1994,1994/1995
20,Oldham,42,9.0,20.0,13.0,42.0,68.0,-26.0,40,21,1993/1994,1994/1995


In [24]:
df = pd.merge(left=df, right=all_results[['Team', 'next_season', 'position']].rename(columns={'position': 'home_prev_pos'}), how='left', left_on=['HomeTeam', 'season'], right_on=['Team', 'next_season']).drop(columns=['Team', 'next_season'])
df = pd.merge(left=df, right=all_results[['Team', 'next_season', 'position']].rename(columns={'position': 'away_prev_pos'}), how='left', left_on=['AwayTeam', 'season'], right_on=['Team', 'next_season']).drop(columns=['Team', 'next_season'])
df

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,home_prev_pos,away_prev_pos
0,2020/2021,2020-09-12 00:00:00,Fulham,Arsenal,0.0,3.0,NaN,8.0
1,2020/2021,2020-09-12 00:00:00,Crystal Palace,Southampton,1.0,0.0,14.0,11.0
2,2020/2021,2020-09-12 00:00:00,Liverpool,Leeds,4.0,3.0,1.0,NaN
3,2020/2021,2020-09-12 00:00:00,West Ham,Newcastle,0.0,2.0,16.0,13.0
4,2020/2021,2020-09-13 00:00:00,West Brom,Leicester,0.0,3.0,NaN,5.0
...,...,...,...,...,...,...,...,...
10367,1993/1994,1994-05-07 00:00:00,Sheffield Weds,Man City,1.0,1.0,NaN,NaN
10368,1993/1994,1994-05-07 00:00:00,Swindon,Leeds,0.0,5.0,NaN,NaN
10369,1993/1994,1994-05-07 00:00:00,Tottenham,QPR,1.0,2.0,NaN,NaN
10370,1993/1994,1994-05-07 00:00:00,West Ham,Southampton,3.0,3.0,NaN,NaN


In [25]:
df = df[df.season != seasons[-1]]
df['home_prev_pos'] = df['home_prev_pos'].fillna('P')
df['away_prev_pos'] = df['away_prev_pos'].fillna('P')
df

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,home_prev_pos,away_prev_pos
0,2020/2021,2020-09-12 00:00:00,Fulham,Arsenal,0.0,3.0,P,8
1,2020/2021,2020-09-12 00:00:00,Crystal Palace,Southampton,1.0,0.0,14,11
2,2020/2021,2020-09-12 00:00:00,Liverpool,Leeds,4.0,3.0,1,P
3,2020/2021,2020-09-12 00:00:00,West Ham,Newcastle,0.0,2.0,16,13
4,2020/2021,2020-09-13 00:00:00,West Brom,Leicester,0.0,3.0,P,5
...,...,...,...,...,...,...,...,...
9905,1994/1995,1995-05-14 00:00:00,Norwich,Aston Villa,1.0,1.0,12,10
9906,1994/1995,1995-05-14 00:00:00,Sheffield Weds,Ipswich,4.0,1.0,7,19
9907,1994/1995,1995-05-14 00:00:00,Southampton,Leicester,2.0,2.0,18,P
9908,1994/1995,1995-05-14 00:00:00,Tottenham,Leeds,1.0,1.0,15,5


In [26]:
df = df[(df.home_prev_pos == 1) | (df.away_prev_pos == 1)]
df = df[(df.home_prev_pos == 'P') | (df.away_prev_pos == 'P')]
df

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,home_prev_pos,away_prev_pos
2,2020/2021,2020-09-12 00:00:00,Liverpool,Leeds,4.0,3.0,1,P
84,2019/2020,2019-09-14 00:00:00,Norwich,Man City,3.0,2.0,P,1
129,2019/2020,2019-10-26 00:00:00,Man City,Aston Villa,3.0,0.0,1,P
236,2019/2020,2019-12-29 00:00:00,Man City,Sheffield United,2.0,0.0,1,P
256,2019/2020,2020-01-12 00:00:00,Aston Villa,Man City,1.0,6.0,P,1
...,...,...,...,...,...,...,...,...
9602,1994/1995,1994-11-19 00:00:00,Man United,Crystal Palace,3.0,0.0,1,P
9652,1994/1995,1994-12-17 00:00:00,Man United,Nott'm Forest,1.0,2.0,1,P
9673,1994/1995,1994-12-28 00:00:00,Man United,Leicester,1.0,1.0,1,P
9721,1994/1995,1995-01-25 00:00:00,Crystal Palace,Man United,1.0,1.0,P,1


In [27]:
df['goal_diff'] = np.abs(df['FTHG'] - df['FTAG'])

In [28]:
df['champ_win'] = np.where((df.home_prev_pos == 1) & (df.FTHG > df.FTAG), True, False)
df['champ_win'] = np.where((df.away_prev_pos == 1) & (df.FTAG > df.FTHG), True, df['champ_win'])

In [29]:
df

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,home_prev_pos,away_prev_pos,goal_diff,champ_win
2,2020/2021,2020-09-12 00:00:00,Liverpool,Leeds,4.0,3.0,1,P,1.0,True
84,2019/2020,2019-09-14 00:00:00,Norwich,Man City,3.0,2.0,P,1,1.0,False
129,2019/2020,2019-10-26 00:00:00,Man City,Aston Villa,3.0,0.0,1,P,3.0,True
236,2019/2020,2019-12-29 00:00:00,Man City,Sheffield United,2.0,0.0,1,P,2.0,True
256,2019/2020,2020-01-12 00:00:00,Aston Villa,Man City,1.0,6.0,P,1,5.0,True
...,...,...,...,...,...,...,...,...,...,...
9602,1994/1995,1994-11-19 00:00:00,Man United,Crystal Palace,3.0,0.0,1,P,3.0,True
9652,1994/1995,1994-12-17 00:00:00,Man United,Nott'm Forest,1.0,2.0,1,P,1.0,False
9673,1994/1995,1994-12-28 00:00:00,Man United,Leicester,1.0,1.0,1,P,0.0,False
9721,1994/1995,1995-01-25 00:00:00,Crystal Palace,Man United,1.0,1.0,P,1,0.0,False


In [30]:
df[(~df.champ_win) & (df.FTHG != df.FTAG)].sort_values(['goal_diff', 'Date'], ascending=[False, False]).head(30)

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,home_prev_pos,away_prev_pos,goal_diff,champ_win
1173,2017/2018,2018-05-13 00:00:00,Newcastle,Chelsea,3.0,0.0,P,1,3.0,False
2950,2012/2013,2013-02-09 00:00:00,Southampton,Man City,3.0,1.0,P,1,2.0,False
9142,1995/1996,1995-09-30 00:00:00,Middlesbrough,Blackburn,2.0,0.0,P,1,2.0,False
84,2019/2020,2019-09-14 00:00:00,Norwich,Man City,3.0,2.0,P,1,1.0,False
1400,2016/2017,2017-01-31 00:00:00,Burnley,Leicester,1.0,0.0,P,1,1.0,False
1181,2016/2017,2016-08-13 00:00:00,Hull,Leicester,2.0,1.0,P,1,1.0,False
1699,2015/2016,2015-12-05 00:00:00,Chelsea,Bournemouth,0.0,1.0,1,P,1.0,False
2218,2014/2015,2015-03-14 00:00:00,Burnley,Man City,1.0,0.0,P,1,1.0,False
3851,2009/2010,2009-08-19 00:00:00,Burnley,Man United,1.0,0.0,P,1,1.0,False
6287,2003/2004,2004-01-17 00:00:00,Wolves,Man United,1.0,0.0,P,1,1.0,False


In [31]:
# now we get the promoted and champions from the previous season
promoted = {}
prev_champion = {}

for season, table in league_tables.items():
    if list(seasons).index(season) < len(seasons)-1:
        prev_season = seasons[list(seasons).index(season) +1]
        p = [x for x in table.index if x not in league_tables[prev_season].index]
        promoted[season] = p
        

In [32]:
promoted

{'2020/2021': [],
 '2019/2020': [],
 '2018/2019': [],
 '2017/2018': [],
 '2016/2017': [],
 '2015/2016': [],
 '2014/2015': [],
 '2013/2014': [],
 '2012/2013': [],
 '2011/2012': [],
 '2010/2011': [],
 '2009/2010': [],
 '2008/2009': [],
 '2007/2008': [],
 '2006/2007': [],
 '2005/2006': [],
 '2004/2005': [],
 '2003/2004': [],
 '2002/2003': [],
 '2001/2002': [],
 '2000/2001': [],
 '1999/1900': [],
 '1998/1999': [],
 '1997/1998': [],
 '1996/1997': [],
 '1995/1996': [],
 '1994/1995': []}

In [33]:
seasons = df.season.unique()
div = 'E0'
league_tables = {}
conn = sqlite3.connect(DATA_PATH)

for s in seasons:
    
    x = league_table_asof(div, s, None, conn)
    league_tables[s] = x

In [34]:
champions = {}

for season, table in league_tables.items():
    champions[season] = table.index[0]

In [35]:
prev_champions = {}

for season,champ in champions.items():
    ind = list(seasons).index(season)
    if ind == 0:
        pass
    else:
        prev_champions[seasons[ind-1]] = [champ, seasons[ind]]

prev_champions

{'2020/2021': ['Liverpool', '2019/2020'],
 '2019/2020': ['Man City', '2018/2019'],
 '2018/2019': ['Man City', '2017/2018'],
 '2017/2018': ['Chelsea', '2016/2017'],
 '2016/2017': ['Leicester', '2015/2016'],
 '2015/2016': ['Chelsea', '2014/2015'],
 '2014/2015': ['Man City', '2013/2014'],
 '2013/2014': ['Man United', '2012/2013'],
 '2012/2013': ['Man City', '2011/2012'],
 '2011/2012': ['Man United', '2010/2011'],
 '2010/2011': ['Chelsea', '2009/2010'],
 '2009/2010': ['Man United', '2008/2009'],
 '2008/2009': ['Man United', '2007/2008'],
 '2007/2008': ['Man United', '2006/2007'],
 '2006/2007': ['Chelsea', '2005/2006'],
 '2005/2006': ['Chelsea', '2004/2005'],
 '2004/2005': ['Arsenal', '2003/2004'],
 '2003/2004': ['Man United', '2002/2003'],
 '2002/2003': ['Arsenal', '2001/2002'],
 '2001/2002': ['Man United', '2000/2001'],
 '2000/2001': ['Man United', '1999/1900'],
 '1999/1900': ['Man United', '1998/1999'],
 '1998/1999': ['Arsenal', '1997/1998'],
 '1997/1998': ['Man United', '1996/1997'],
 '

In [36]:
champ_results = []

for season, champ in prev_champions.items():
    df_cr = create_and_query('matches', ['season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG' ,'FTAG', 'FTR'], {'Div': ['=', 'E0'], 'season': ['=', season]}).dropna()
    
    df_cr = df_cr[(df_cr.HomeTeam == champ[0]) | (df_cr.AwayTeam == champ[0])]
    
    df_cr['champ'] = champ[0]
    df_cr['prev_season'] = champ[1]
    
    champ_results.append(df_cr)

Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2020/2021'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2019/2020'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2018/2019'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2017/2018'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2016/2017'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2015/2016'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND season = '2014/2015'
Running query: SELECT season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR FROM matches WHERE Div = 'E0' AND seaso

In [37]:
res = pd.concat(champ_results)

In [38]:
res['champ_win'] = np.where((res.HomeTeam == res.champ) & (res.FTR == 'H'), True, False)
res['champ_win'] = np.where((res.AwayTeam == res.champ) & (res.FTR == 'A'), True, res.champ_win)
res.head(10)

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,champ,prev_season,champ_win
2,2020/2021,2020-09-12 00:00:00,Liverpool,Leeds,4.0,3.0,H,Liverpool,2019/2020,True
14,2020/2021,2020-09-20 00:00:00,Chelsea,Liverpool,0.0,2.0,A,Liverpool,2019/2020,True
27,2020/2021,2020-09-28 00:00:00,Liverpool,Arsenal,3.0,1.0,H,Liverpool,2019/2020,True
37,2020/2021,2020-10-04 00:00:00,Aston Villa,Liverpool,7.0,2.0,H,Liverpool,2019/2020,False
1,2019/2020,2019-08-10 00:00:00,West Ham,Man City,0.0,5.0,A,Man City,2018/2019,True
16,2019/2020,2019-08-17 00:00:00,Man City,Tottenham,2.0,2.0,D,Man City,2018/2019,False
27,2019/2020,2019-08-25 00:00:00,Bournemouth,Man City,1.0,3.0,A,Man City,2018/2019,True
34,2019/2020,2019-08-31 00:00:00,Man City,Brighton,4.0,0.0,H,Man City,2018/2019,True
46,2019/2020,2019-09-14 00:00:00,Norwich,Man City,3.0,2.0,H,Man City,2018/2019,False
54,2019/2020,2019-09-21 00:00:00,Man City,Watford,8.0,0.0,H,Man City,2018/2019,True


In [39]:
res = res[res.champ_win == False]
res

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,champ,prev_season,champ_win
37,2020/2021,2020-10-04 00:00:00,Aston Villa,Liverpool,7.0,2.0,H,Liverpool,2019/2020,False
16,2019/2020,2019-08-17 00:00:00,Man City,Tottenham,2.0,2.0,D,Man City,2018/2019,False
46,2019/2020,2019-09-14 00:00:00,Norwich,Man City,3.0,2.0,H,Man City,2018/2019,False
77,2019/2020,2019-10-06 00:00:00,Man City,Wolves,0.0,2.0,A,Man City,2018/2019,False
119,2019/2020,2019-11-10 00:00:00,Liverpool,Man City,3.0,1.0,H,Man City,2018/2019,False
...,...,...,...,...,...,...,...,...,...,...
271,1995/1996,1996-02-28 00:00:00,Aston Villa,Blackburn,2.0,0.0,H,Blackburn,1994/1995,False
274,1995/1996,1996-03-02 00:00:00,Man City,Blackburn,1.0,1.0,D,Blackburn,1994/1995,False
313,1995/1996,1996-03-30 00:00:00,Blackburn,Everton,0.0,3.0,A,Blackburn,1994/1995,False
329,1995/1996,1996-04-06 00:00:00,Southampton,Blackburn,1.0,0.0,H,Blackburn,1994/1995,False


In [40]:
res['loss_GD'] = np.abs(res.FTHG - res.FTAG)
res = res[res.FTR != 'D']
res.head(10)

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,champ,prev_season,champ_win,loss_GD
37,2020/2021,2020-10-04 00:00:00,Aston Villa,Liverpool,7.0,2.0,H,Liverpool,2019/2020,False,5.0
46,2019/2020,2019-09-14 00:00:00,Norwich,Man City,3.0,2.0,H,Man City,2018/2019,False,1.0
77,2019/2020,2019-10-06 00:00:00,Man City,Wolves,0.0,2.0,A,Man City,2018/2019,False,2.0
119,2019/2020,2019-11-10 00:00:00,Liverpool,Man City,3.0,1.0,H,Man City,2018/2019,False,2.0
154,2019/2020,2019-12-07 00:00:00,Man City,Man United,1.0,2.0,A,Man City,2018/2019,False,1.0
188,2019/2020,2019-12-27 00:00:00,Wolves,Man City,3.0,2.0,H,Man City,2018/2019,False,1.0
249,2019/2020,2020-02-02 00:00:00,Tottenham,Man City,2.0,0.0,H,Man City,2018/2019,False,2.0
286,2019/2020,2020-03-08 00:00:00,Man United,Man City,2.0,0.0,H,Man City,2018/2019,False,2.0
309,2019/2020,2020-06-25 00:00:00,Chelsea,Man City,2.0,1.0,H,Man City,2018/2019,False,1.0
328,2019/2020,2020-07-05 00:00:00,Southampton,Man City,1.0,0.0,H,Man City,2018/2019,False,1.0


In [41]:
res.sort_values('loss_GD', ascending=False).head(10)

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,champ,prev_season,champ_win,loss_GD
37,2020/2021,2020-10-04 00:00:00,Aston Villa,Liverpool,7.0,2.0,H,Liverpool,2019/2020,False,5.0
161,1995/1996,1995-12-09 00:00:00,Coventry,Blackburn,5.0,0.0,H,Blackburn,1994/1995,False,5.0
87,2011/2012,2011-10-23 00:00:00,Man United,Man City,1.0,6.0,A,Man United,2010/2011,False,5.0
369,2016/2017,2017-05-18 00:00:00,Leicester,Tottenham,1.0,6.0,A,Leicester,2015/2016,False,5.0
98,1996/1997,1996-10-20 00:00:00,Newcastle,Man United,5.0,0.0,H,Man United,1995/1996,False,5.0
90,1999/1900,1999-10-03 00:00:00,Chelsea,Man United,5.0,0.0,H,Man United,1998/1999,False,5.0
249,2005/2006,2006-02-11 00:00:00,Middlesbrough,Chelsea,3.0,0.0,H,Chelsea,2004/2005,False,3.0
219,2016/2017,2017-01-22 00:00:00,Southampton,Leicester,3.0,0.0,H,Leicester,2015/2016,False,3.0
53,2016/2017,2016-09-24 00:00:00,Man United,Leicester,4.0,1.0,H,Leicester,2015/2016,False,3.0
72,2016/2017,2016-10-15 00:00:00,Chelsea,Leicester,3.0,0.0,H,Leicester,2015/2016,False,3.0
